In [3]:
# enhanced_formal_assistant.py
import os
import speech_recognition as sr
import pygame
import tempfile
import time
from gtts import gTTS
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import re
import random
from datetime import datetime

# --- Configuration ---
KB_INDEX_PATH = "faiss_index_contextual"
MAX_CHUNK_LENGTH = 150

class EnhancedFormalAssistant:
    def __init__(self):
        pygame.mixer.init()
        self.vector_store = self.load_knowledge_base()
        self.user_name = None
        self.conversation_count = 0
        
    def load_knowledge_base(self):
        """Load the FAISS knowledge base"""
        try:
            embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
            if os.path.exists(KB_INDEX_PATH):
                vector_store = FAISS.load_local(KB_INDEX_PATH, embeddings, allow_dangerous_deserialization=True)
                print("✅ Knowledge base loaded")
                return vector_store
            else:
                print("❌ Knowledge base not found")
                return None
        except Exception as e:
            print(f"Error loading knowledge base: {e}")
            return None

    # --- Formal Responses Database ---
    GREETINGS = {
        'hello': [
            "Hello! It's wonderful to speak with you. How may I assist you today?",
            "Greetings! I'm here to help with information about brain tumors and medical topics. What would you like to know?",
            "Good day! I'm your medical information assistant. How can I be of service?",
            "Hello there! I'm ready to provide you with helpful medical information. What's on your mind?"
        ],
        'hi': [
            "Hi! Lovely to meet you. How can I help you today?",
            "Hi there! I'm here to assist with medical inquiries. What would you like to discuss?",
            "Hello! I'm your friendly medical assistant. What can I help you with?"
        ],
        'good_morning': [
            "Good morning! A wonderful day to seek knowledge. How may I assist you?",
            "Morning! I hope you're having a great start to your day. What medical information can I provide?",
            "Good morning! I'm here to help with any health-related questions you might have."
        ],
        'good_afternoon': [
            "Good afternoon! I hope your day is going well. How can I assist you?",
            "Afternoon! Ready to provide you with helpful medical information. What do you need?",
            "Good afternoon! What health topic would you like to explore today?"
        ],
        'good_evening': [
            "Good evening! I'm here to help with your medical queries, even at this hour.",
            "Evening! How can I assist you with health information tonight?",
            "Good evening! What brings you to our medical assistant today?"
        ]
    }

    APPRECIATION_RESPONSES = {
        'thanks': [
            "You're most welcome! I'm glad I could assist you.",
            "My pleasure! Don't hesitate to ask if you need more information.",
            "You're very welcome! I'm here whenever you need medical guidance.",
            "Happy to help! Is there anything else you'd like to know?"
        ],
        'thank_you': [
            "You're absolutely welcome! It's my purpose to provide helpful information.",
            "The pleasure is mine! Feel free to ask any other medical questions.",
            "You're very welcome! I'm here to support your health information needs."
        ],
        'appreciation': [
            "I appreciate your kind words! It motivates me to be more helpful.",
            "Thank you for your appreciation! I'm dedicated to providing accurate information.",
            "That's very kind of you to say! I'm here whenever you need assistance."
        ]
    }

    POLITE_RESPONSES = {
        'how_are_you': [
            "I'm functioning well, thank you for asking! Ready to assist with your medical queries.",
            "I'm doing great! Eager to help you with health information.",
            "I'm operating perfectly! How can I assist you today?"
        ],
        'who_are_you': [
            "I'm your Medical Information Assistant, specialized in providing information about brain tumors and related health topics.",
            "I'm an AI assistant designed to help with medical information, particularly focused on brain tumor education and support.",
            "I'm your dedicated health information assistant, here to provide reliable information about brain tumors and neurological health."
        ],
        'what_can_you_do': [
            "I can provide comprehensive information about brain tumors, including symptoms, diagnosis, treatment options, and support resources. I can also answer general medical questions and offer guidance.",
            "I specialize in brain tumor information but can also assist with general health queries. I provide explanations about medical terms, treatment options, and support available for patients and caregivers.",
            "My expertise includes detailed information about brain tumors, medical procedures, symptom analysis, and healthcare guidance. I'm here to educate and support your health journey."
        ]
    }

    FAREWELLS = [
        "Goodbye! Thank you for using our medical assistant. Take care of your health!",
        "Farewell! Remember that early detection and proper medical care are crucial for health.",
        "Goodbye! I hope the information was helpful. Wishing you the best in your health journey.",
        "Take care! Don't hesitate to return if you have more medical questions.",
        "Until next time! Remember to consult healthcare professionals for personalized medical advice."
    ]

    UNABLE_TO_ANSWER = [
        "I apologize, but I don't have specific information about that in my medical knowledge base. Could you try rephrasing your question?",
        "That's outside my current medical expertise. I'm primarily focused on brain tumor information and related health topics.",
        "I'm not equipped to answer that specific medical question. I specialize in brain tumor education and general health information.",
        "My knowledge is limited to medical topics, particularly brain tumors. Could you ask about health-related subjects?"
    ]

    def get_greeting_response(self, query):
        """Handle greeting queries"""
        query_lower = query.lower()
        
        if any(word in query_lower for word in ['hello', 'hi', 'hey']):
            return random.choice(self.GREETINGS['hello'])
        elif 'good morning' in query_lower:
            return random.choice(self.GREETINGS['good_morning'])
        elif 'good afternoon' in query_lower:
            return random.choice(self.GREETINGS['good_afternoon'])
        elif 'good evening' in query_lower:
            return random.choice(self.GREETINGS['good_evening'])
        
        return None

    def get_appreciation_response(self, query):
        """Handle thank you and appreciation"""
        query_lower = query.lower()
        
        if any(word in query_lower for word in ['thank you', 'thanks', 'thankyou']):
            return random.choice(self.APPRECIATION_RESPONSES['thanks'])
        elif any(word in query_lower for word in ['appreciate', 'grateful', 'nice help']):
            return random.choice(self.APPRECIATION_RESPONSES['appreciation'])
        
        return None

    def get_polite_response(self, query):
        """Handle polite conversation"""
        query_lower = query.lower()
        
        if any(phrase in query_lower for phrase in ['how are you', "how're you", "how do you do"]):
            return random.choice(self.POLITE_RESPONSES['how_are_you'])
        elif any(phrase in query_lower for phrase in ['who are you', 'what are you']):
            return random.choice(self.POLITE_RESPONSES['who_are_you'])
        elif any(phrase in query_lower for phrase in ['what can you do', 'what do you do', 'your purpose']):
            return random.choice(self.POLITE_RESPONSES['what_can_you_do'])
        elif 'your name' in query_lower:
            return "I'm your Medical Information Assistant. You can think of me as your dedicated health companion."
        
        return None

    def get_farewell_response(self):
        """Get a farewell message"""
        return random.choice(self.FAREWELLS)

    def extract_name(self, query):
        """Extract name from query if provided"""
        query_lower = query.lower()
        name_indicators = ['my name is', "i'm", 'im ', 'call me', 'name is']
        
        for indicator in name_indicators:
            if indicator in query_lower:
                # Extract the name part
                name_part = query_lower.split(indicator, 1)[1].strip()
                # Take first word as name
                name = name_part.split()[0].title()
                return name
        return None

    def split_text_into_chunks(self, text):
        """Split text into natural speaking chunks"""
        text = re.sub(r'\s+', ' ', text).strip()
        sentences = re.split(r'[.!?]+', text)
        sentences = [s.strip() for s in sentences if s.strip()]
        
        chunks = []
        current_chunk = ""
        
        for sentence in sentences:
            if len(current_chunk) + len(sentence) + 1 > MAX_CHUNK_LENGTH and current_chunk:
                chunks.append(current_chunk.strip())
                current_chunk = sentence
            else:
                if current_chunk:
                    current_chunk += ". " + sentence
                else:
                    current_chunk = sentence
        
        if current_chunk:
            chunks.append(current_chunk.strip())
        
        if not chunks:
            chunks = [text[i:i+MAX_CHUNK_LENGTH] for i in range(0, len(text), MAX_CHUNK_LENGTH)]
        
        return chunks

    def speak_chunk(self, text_chunk):
        """Speak a single chunk of text"""
        try:
            if not text_chunk or len(text_chunk.strip()) == 0:
                return True
            
            tts = gTTS(text=text_chunk, lang='en', slow=False)
            
            with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as tmp_file:
                temp_path = tmp_file.name
            
            tts.save(temp_path)
            
            pygame.mixer.music.load(temp_path)
            pygame.mixer.music.play()
            
            while pygame.mixer.music.get_busy():
                pygame.time.wait(100)
            
            pygame.mixer.music.unload()
            if os.path.exists(temp_path):
                os.unlink(temp_path)
            
            time.sleep(0.3)
            return True
            
        except Exception as e:
            print(f"❌ TTS error: {e}")
            if 'temp_path' in locals() and os.path.exists(temp_path):
                try:
                    pygame.mixer.music.unload()
                    os.unlink(temp_path)
                except:
                    pass
            return False

    def speak(self, text):
        """Convert text to speech using chunking"""
        try:
            print(f"🤖 Assistant: {text}")
            
            chunks = self.split_text_into_chunks(text)
            
            if len(chunks) > 1:
                print(f"🗣️  Speaking {len(chunks)} parts...")
            
            successful_chunks = 0
            
            for i, chunk in enumerate(chunks):
                if self.speak_chunk(chunk):
                    successful_chunks += 1
                else:
                    print(f"❌ Failed to speak part {i+1}")
                    continue
            
            return successful_chunks > 0
            
        except Exception as e:
            print(f"🎯 TTS error: {e}")
            return False

    def listen(self):
        """Listen to user speech"""
        recognizer = sr.Recognizer()
        
        try:
            with sr.Microphone() as source:
                print("\n🎤 Listening... (speak now)")
                recognizer.adjust_for_ambient_noise(source, duration=0.5)
                audio = recognizer.listen(source, timeout=10, phrase_time_limit=8)
            
            print("🧠 Processing...")
            text = recognizer.recognize_google(audio)
            print(f"👤 User: {text}")
            return text.lower()
            
        except sr.WaitTimeoutError:
            print("⏰ Listening timeout")
            return "timeout"
        except sr.UnknownValueError:
            print("❓ Could not understand audio")
            self.speak("I apologize, I didn't catch that. Could you please repeat your question?")
            return "unknown"
        except Exception as e:
            print(f"🎤 Microphone error: {e}")
            return "error"

    def process_formal_query(self, query):
        """Process formal and conversational queries"""
        self.conversation_count += 1
        
        # Check for name introduction
        if not self.user_name:
            extracted_name = self.extract_name(query)
            if extracted_name:
                self.user_name = extracted_name
                return f"Nice to meet you, {self.user_name}! How can I assist you today?"
        
        # Use name in response if available
        name_suffix = f", {self.user_name}" if self.user_name else ""
        
        # Check different query types
        response = self.get_greeting_response(query)
        if response:
            return response + name_suffix if name_suffix and 'hello' in query.lower() else response
        
        response = self.get_appreciation_response(query)
        if response:
            return response
        
        response = self.get_polite_response(query)
        if response:
            return response
        
        # Special cases
        query_lower = query.lower()
        
        if any(word in query_lower for word in ['yes', 'yeah', 'yep', 'correct']):
            return "Great! What would you like to know next?"
        
        elif any(word in query_lower for word in ['no', 'nope', 'not really']):
            return "I understand. Is there something else I can help you with?"
        
        elif 'i don' in query_lower and 'know' in query_lower:
            return "That's perfectly alright. I'm here to help you understand. What specific information are you looking for?"
        
        elif any(word in query_lower for word in ['help', 'support', 'assist']):
            return "I'm here to help! I can provide information about brain tumors, symptoms, treatments, and general health guidance. What specifically would you like to know?"
        
        elif any(word in query_lower for word in ['sorry', 'apologize', 'apologies']):
            return "No need to apologize! I'm here to help without judgment. What can I assist you with?"
        
        return None

    def get_medical_response(self, query):
        """Generate medical response from knowledge base"""
        if not self.vector_store:
            return "I apologize, but my medical knowledge base is currently unavailable. Please try again later."
        
        try:
            docs = self.vector_store.similarity_search(query, k=1)
            
            if not docs:
                return random.choice(self.UNABLE_TO_ANSWER)
            
            response = docs[0].page_content.strip()
            response = re.sub(r'\s+', ' ', response)
            
            # Add polite introduction
            polite_intros = [
                "Based on medical information,",
                "According to healthcare knowledge,",
                "Medical sources indicate that",
                "Here's what I can share about that:"
            ]
            
            intro = random.choice(polite_intros)
            return f"{intro} {response}"
            
        except Exception as e:
            print(f"Response error: {e}")
            return "I apologize, but I encountered an error while retrieving medical information. Please try again."

    def get_response(self, query):
        """Main response handler"""
        # First, check for formal/conversational queries
        formal_response = self.process_formal_query(query)
        if formal_response:
            return formal_response
        
        # Then, check for exit commands
        if any(word in query.lower() for word in ['exit', 'quit', 'stop', 'goodbye', 'bye']):
            return self.get_farewell_response()
        
        # Finally, get medical response
        return self.get_medical_response(query)

    def run(self):
        """Main assistant loop"""
        print("\n" + "="*50)
        print("🏥 ENHANCED MEDICAL VOICE ASSISTANT STARTING...")
        print("="*50)
        
        if not self.vector_store:
            print("❌ Knowledge base failed to load.")
            return
        
        # Initial greeting based on time of day
        current_hour = datetime.now().hour
        if 5 <= current_hour < 12:
            greeting = "Good morning! I'm your Medical Information Assistant."
        elif 12 <= current_hour < 17:
            greeting = "Good afternoon! I'm your Medical Information Assistant."
        elif 17 <= current_hour < 22:
            greeting = "Good evening! I'm your Medical Information Assistant."
        else:
            greeting = "Hello! I'm your Medical Information Assistant, available even during late hours."
        
        greeting += " I specialize in brain tumor information and general health guidance. How may I assist you today?"
        
        self.speak(greeting)
        
        while True:
            try:
                user_input = self.listen()
                
                if user_input in ["timeout", "unknown", "error"]:
                    if user_input == "timeout" and self.conversation_count > 0:
                        self.speak("I notice you've been quiet. Are you still there? Feel free to ask any medical questions.")
                    continue
                
                response = self.get_response(user_input)
                self.speak(response)
                
                # Check if it was a farewell response
                if any(word in response.lower() for word in ['goodbye', 'farewell', 'take care']):
                    break
                
            except KeyboardInterrupt:
                print("\n🛑 Assistant stopped by user")
                self.speak("Session ended. Thank you for using our medical assistant!")
                break
            except Exception as e:
                print(f"Unexpected error: {e}")

# Run the assistant
if __name__ == "__main__":
    assistant = EnhancedFormalAssistant()
    assistant.run()

✅ Knowledge base loaded

🏥 ENHANCED MEDICAL VOICE ASSISTANT STARTING...
🤖 Assistant: Good evening! I'm your Medical Information Assistant. I specialize in brain tumor information and general health guidance. How may I assist you today?

🎤 Listening... (speak now)
🧠 Processing...
👤 User: hu r u
🤖 Assistant: Here's what I can share about that: Novel drug delivery strategies aim to overcome the blood-brain barrier, including convection-enhanced delivery, nanoparticle carriers, and focused ultrasound disruption. Immunotherapy combinations seek to overcome the immunosuppressive tumor microenvironment. Gene therapy approaches investigate viral vectors and CRISPR-based technologies for targeted interventions.
🗣️  Speaking 3 parts...

🛑 Assistant stopped by user
🤖 Assistant: Session ended. Thank you for using our medical assistant!


In [ ]:
# # audio_diagnostic.py
# import pyttsx3
# import speech_recognition as sr
# import pyaudio

# def test_audio_system():
#     print("🔊 Running Audio System Diagnostics...\n")
    
#     # Test 1: Check PyAudio (for microphone)
#     print("1. Testing PyAudio (Microphone)...")
#     try:
#         p = pyaudio.PyAudio()
#         print(f"   ✅ PyAudio initialized. Found {p.get_device_count()} audio devices:")
        
#         for i in range(p.get_device_count()):
#             device_info = p.get_device_info_by_index(i)
#             print(f"      Device {i}: {device_info['name']} - {device_info['maxInputChannels']} in / {device_info['maxOutputChannels']} out")
        
#         p.terminate()
#     except Exception as e:
#         print(f"   ❌ PyAudio error: {e}")
    
#     # Test 2: Check Text-to-Speech
#     print("\n2. Testing Text-to-Speech...")
#     try:
#         engine = pyttsx3.init()
#         voices = engine.getProperty('voices')
#         print(f"   ✅ TTS Engine initialized. Found {len(voices)} voices:")
        
#         for i, voice in enumerate(voices):
#             print(f"      Voice {i}: {voice.name}")
        
#         # Test actual speech
#         print("   🗣️  Testing speech output...")
#         engine.say("This is a test message. Can you hear me?")
#         engine.runAndWait()
#         print("   ✅ Speech test completed")
        
#     except Exception as e:
#         print(f"   ❌ TTS error: {e}")
    
#     # Test 3: Check Speech Recognition
#     print("\n3. Testing Speech Recognition...")
#     try:
#         r = sr.Recognizer()
#         with sr.Microphone() as source:
#             print("   🎤 Please speak something after the beep...")
#             r.adjust_for_ambient_noise(source, duration=1)
#             print("   🔊 Listening...")
#             audio = r.listen(source, timeout=5)
        
#         print("   ✅ Audio captured successfully")
#     except Exception as e:
#         print(f"   ❌ Speech recognition error: {e}")
    
#     print("\n🎯 Diagnostic complete!")

# if __name__ == "__main__":
#     test_audio_system()

🔊 Running Audio System Diagnostics...

1. Testing PyAudio (Microphone)...
   ✅ PyAudio initialized. Found 24 audio devices:
      Device 0: Microsoft Sound Mapper - Input - 2 in / 0 out
      Device 1: Microphone Array (AMD Audio Dev - 2 in / 0 out
      Device 2: Microsoft Sound Mapper - Output - 0 in / 2 out
      Device 3: Speaker (Realtek(R) Audio) - 0 in / 2 out
      Device 4: LG FULL HD (NVIDIA High Definit - 0 in / 2 out
      Device 5: Primary Sound Capture Driver - 2 in / 0 out
      Device 6: Microphone Array (AMD Audio Device) - 2 in / 0 out
      Device 7: Primary Sound Driver - 0 in / 2 out
      Device 8: Speaker (Realtek(R) Audio) - 0 in / 2 out
      Device 9: LG FULL HD (NVIDIA High Definition Audio) - 0 in / 2 out
      Device 10: Speaker (Realtek(R) Audio) - 0 in / 2 out
      Device 11: LG FULL HD (NVIDIA High Definition Audio) - 0 in / 2 out
      Device 12: Microphone Array (AMD Audio Device) - 2 in / 0 out
      Device 13: Microphone (Realtek HD Audio Mic input)